#### [Naver 리뷰 데이터셋 전처리]
<hr>

- train :  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
- test :  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

In [16]:
# 데이터 다운로드
from urllib.request import urlretrieve

filename = '../DATA/ratings_train.txt'

urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', filename)

filename = '../DATA/ratings_test.txt'

urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', filename)


('../DATA/ratings_test.txt', <http.client.HTTPMessage at 0x7d5cb7211e50>)

In [17]:
# 데이터 로드

import pandas as pd

train_data = pd.read_csv('../DATA/ratings_train.txt', delimiter='\t')
test_data = pd.read_csv('../DATA/ratings_test.txt', delimiter='\t')
train_data.head(), test_data.head()

(         id                                           document  label
 0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
 1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
 2  10265843                                  너무재밓었다그래서보는것을추천한다      0
 3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
 4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1,
         id                                           document  label
 0  6270596                                                굳 ㅋ      1
 1  9274899                               GDNTOPCLASSINTHECLUB      0
 2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
 3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
 4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0)

#### (2) 데이터 전처리
<hr>

- 결측치, 중복데이터
- 클래스의 균형 데이터 여부
- 텍스트 데이터 전처리 => 정제 (불용어, 노이즈데이터 제거), 토큰화, 단어사전
- 텍스트 데이터 인코딩
- 텍스트 데이터 패딩


In [18]:
train_data.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [19]:
import konlpy.tag as Tag
tagger = Tag.Mecab()



In [20]:

morphs0 = [tagger.nouns(x) for x in train_data[train_data['label'] == 0].document.to_list() if isinstance(x, str)]
morphs1 = [tagger.nouns(x) for x in train_data[train_data['label'] == 1].document.to_list() if isinstance(x, str)]
pos0 = [tagger.pos(x) for x in train_data[train_data['label'] == 0].document.to_list() if isinstance(x, str)]
pos1 = [tagger.pos(x) for x in train_data[train_data['label'] == 1].document.to_list() if isinstance(x, str)]

result0 = []
result1 = []
resultpos0 = []
resultpos1 = []
def oneD(morphs0, result0):
    for idx, x in enumerate(morphs0):
        result0 += x

oneD(morphs0, result0)
oneD(morphs1, result1)

oneD(pos0, resultpos0)
oneD(pos1, resultpos1)


#### (2-2) 텍스트 데이터 전처리 : 정제 (노이즈 데이터, 불용어 제거), 토큰화, 정수화


In [102]:
###  ===> 노이즈 데이터 기준 : 한글만, 한글+영어, 한글+영어+숫자, 한글+영어+특수문자
import string
import re
from konlpy.tag import Mecab
from copy import deepcopy

# 정규식 적용 regex=True
stop_word = string.punctuation
targetdf = train_data[train_data['label'] == 1]
mecab = Mecab()
stopwords = []
with open('../DATA/stopwords.txt', 'r') as f:
    stopwords = f.read().split('\n')

stopset = set(stopwords)

def returnresult(targetdf):
    global memorphs
    
    hangulregex = re.compile('[^ㄱ-ㅣ가-힣]')
    hangulregex = re.compile('[^ 가-힣]')

    train_data_replace = targetdf['document'].str.replace(pat='^ +', repl='', regex=True)
    train_data_replace_onlyH = train_data_replace.str.replace(pat=hangulregex, repl='', regex=True)
    train_data_replace_onlyH.dropna(inplace=True)

    hangulstop = 'https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt'
    urlretrieve(hangulstop, '../DATA/stopwords.txt')


    memorphs = mecab.morphs(' '.join(train_data_replace_onlyH.to_list()))


    vocab = {}
    for idx in range(train_data_replace_onlyH.shape[0]):
        result = mecab.morphs(train_data_replace_onlyH.iloc[idx])
        for x in result:
            if len(x) >= 2:
                if x not in vocab:
                    vocab[x] = 1
                else:
                    vocab[x] += 1



    for stop in stopset:
        try:
            vocab.pop(stop)
        except:
            # print(stop, 'not found')
            pass

    vocab_freq = {x:y for x,y in vocab.items() if y > 50}    
    return vocab_freq
vocab_freq_dislike = deepcopy(returnresult(train_data[train_data['label'] == 0]))
vocab_freq_like = deepcopy(returnresult(train_data[train_data['label'] == 1]))
vocab_freq = vocab_freq_like

len(vocab_freq)


sorted([(x, y) for x, y in vocab_freq_dislike.items() if y < 100], key=lambda x: x[1], reverse=True)

[('변태', 99),
 ('자막', 99),
 ('야동', 99),
 ('빈약', 99),
 ('실력', 98),
 ('본인', 98),
 ('안습', 98),
 ('뜬금없', 98),
 ('연결', 98),
 ('청소년', 98),
 ('드립니다', 98),
 ('조작', 98),
 ('인정', 98),
 ('열받', 98),
 ('왠지', 97),
 ('몰라도', 97),
 ('젠장', 97),
 ('출연진', 97),
 ('철학', 97),
 ('한국인', 97),
 ('먼저', 97),
 ('좋아하', 97),
 ('반복', 96),
 ('못해', 96),
 ('해야지', 96),
 ('용서', 96),
 ('아쉬움', 96),
 ('바뀌', 96),
 ('만족', 96),
 ('으며', 95),
 ('아쉬운', 95),
 ('말아먹', 95),
 ('도록', 95),
 ('스러움', 95),
 ('과정', 95),
 ('감흥', 95),
 ('는군', 94),
 ('멋있', 94),
 ('그걸', 94),
 ('때리', 94),
 ('대본', 94),
 ('러브', 94),
 ('무시', 94),
 ('반성', 94),
 ('자살', 94),
 ('관심', 94),
 ('아이돌', 94),
 ('떨어진다', 94),
 ('완성도', 94),
 ('부실', 93),
 ('봣는데', 93),
 ('대한민국', 93),
 ('흔한', 93),
 ('자식', 93),
 ('외국', 93),
 ('아냐', 92),
 ('서로', 92),
 ('스케일', 92),
 ('생애', 92),
 ('더러운', 92),
 ('잔뜩', 92),
 ('실제', 91),
 ('듯이', 91),
 ('어떻', 91),
 ('포인트', 91),
 ('우울', 91),
 ('이용', 91),
 ('작위', 91),
 ('퀄리티', 91),
 ('올리', 91),
 ('확실히', 91),
 ('따로', 90),
 ('엄청난', 90),
 ('달리', 90),
 ('새로운', 90)

#### 토큰화

In [103]:
# for x in set(vocab_freq_dislike.keys()).difference(set(vocab_freq_like.keys())):
#     print(x, vocab_freq_dislike[x])

In [108]:
df = pd.DataFrame([(x, mecab.pos(x)[-1][-1],y ) for x, y in  vocab_freq_dislike.items()], columns=['word', 'pos', 'freq'])
df.head()

df[df['pos'].str.startswith('N')].sort_values('freq', ascending=False).head(10)

df[df['word'].isin(list(set(vocab_freq_dislike.keys()).difference(set(vocab_freq_like.keys()))))]

,word,pos,freq
11,반개,NNG,161
12,아까움,VA+ETN,371
16,살려,VV+EC,53
32,드럽,VA,173
33,못해,VV+EC,96
...,...,...,...
1327,대통령,NNG,52
1328,위험,NNG,55
1331,류영화,NNP,57
1332,정상,NNG,65


In [105]:
memorphs = list(set(memorphs))
for x in stopset:
    try:
        memorphs.remove(x)
    except:
        pass
    
memorphs = sorted(memorphs)

In [106]:
for idx, data in enumerate(memorphs[30000:]):
    if idx > 10:
        break
    print(data)


투입
투자
투장
투쟁
투정
투지
투척
투철
투캅스
투표
투피


#### (3) 데이터셋용 단어사전/어휘사전 생성
<hr>

- 최종 사용할 단어 수
- 특별한 의미 문자 추가 : '\<UNK>\', '\<PAD>\'



In [110]:
VOCAB_DICT = {
    0 : '<UNK>',
    1 : '<PAD>',
}

likesorted = [
    x[0] for x in 
    sorted(
        [
            (x, y) for x, y in  vocab_freq_like.items()
        ], 
        key=lambda x: x[1], 
        reverse=True,
        )
    ]

for idx, data in enumerate(likesorted,2):
    VOCAB_DICT[idx] = data



In [118]:
test = ['영화', '최고', '생각', '아나바다']

encoding = []
sentence = []
for tt in test:
    for k, v in VOCAB_DICT.items():
        FLAG = 0
        if v == tt:
            sentence.append(v)
            encoding.append(k)
            FLAG = 1
            break
    
    if FLAG == 0:
        sentence.append(tt)
        encoding.append(-1)


encoding, sentence
#### (4) 데이터셋용 인코딩

([2, 6, 14, -1], ['영화', '최고', '생각', '아나바다'])

In [29]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.normalizers import Lowercase, Sequence, NFD
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import WordPiece as WordPieceDecoder

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Sequence([NFD(), Lowercase()])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train(['./traintext.txt'])
tokenizer.save('./tokenizer.json')